# Calculate aperiodic parameter

Two method of getting aperiodic parameter

## Import related packages

In [3]:
import warnings
warnings.filterwarnings('ignore')
from scuteegfe import Feature
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import mne
from mne.datasets import eegbci
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf

## get motor image data

In [4]:
def get_data_example_motor_image():
    tmin, tmax = -1., 4.
    event_id = dict(hands=2, feet=3)
    subject = 1
    runs = [6, 10, 14]  # motor imagery: hands vs feet
    raw_fnames = eegbci.load_data(subject, runs)
    raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
    eegbci.standardize(raw)  # set channel names
    montage = make_standard_montage('standard_1005')
    raw.set_montage(montage)

    # Apply band-pass filter
    raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

    events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                       exclude='bads')

    # Read epochs (train will be done only between 1 and 2s)
    # Testing will be done with a running classifier
    epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=None, preload=True)
    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2
    data = epochs.get_data()
    print(data.shape)
    print(raw.info['sfreq'])
    return data, labels

In [5]:
data, _ = get_data_example_motor_image()
# 7-channel data are used.
data = data[:,:7,:]

Extracting EDF parameters from C:\Users\15956\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\15956\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\15956\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hammi

In [6]:
n_epcoh, n_channels, n_samples = data.shape
print(f"data.shape: {n_epcoh} epcoh X {n_channels} channels X {n_samples} samples")

data.shape: 45 epcoh X 7 channels X 801 samples


## aperiodic parameter defined by FOOOF

In [7]:
fea1 = Feature(data=data, sfreq=160,selected_funcs=['aperiodic_periodic_offset_exponent_cf'],
                       funcs_params={"aperiodic_periodic_offset_exponent_cf__sfreq":160,
                       "aperiodic_periodic_offset_exponent_cf__n":512,
                                     "aperiodic_periodic_offset_exponent_cf__freq_range":[1,40],
                                     "aperiodic_periodic_offset_exponent_cf__method":"welch"})

### return description

In [8]:
print(fea1)

45(epochs) x 7(channels) x 2(features)
feature names: ['aperiodic_periodic_offset_exponent_cf0'
 'aperiodic_periodic_offset_exponent_cf1']


The Third dimension： [Offset, Exponent]

In [9]:
n_epochs, n_channels, n_features = fea1.features.shape
print(n_epochs, n_channels, n_features)

45 7 2


## The intercept and slope of PSD

In [11]:
fea2 = Feature(data=data, sfreq=160,selected_funcs=['offset_exponent_cf'],
                funcs_params={"offset_exponent_cf__sfreq":160,
                       "offset_exponent_cf__n":512,
                        "offset_exponent_cf__freq_range":[1,40],
                        "offset_exponent_cf__method":"welch"})
print(fea2.features.shape)

(45, 7, 2)


### return description

In [12]:
print(fea2)

45(epochs) x 7(channels) x 2(features)
feature names: ['offset_exponent_cf0' 'offset_exponent_cf1']


The Third dimension：[intercept, slope]

In [13]:
n_epochs, n_channels, n_features = fea1.features.shape
print(n_epochs, n_channels, n_features)

45 7 2
